# Hodgkin-Huxley Model: Action Potential Simulation

**Yêu cầu:** Định nghĩa parameters, 6 hàm tốc độ (α_x, β_x). V₀ = -65 mV, I_ext = 20 μA/cm²  
**Output:** Plot V, I_Na, I_K + Animation + Giải thích depolarization/repolarization

## Setup: Import libraries

In [8]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from scipy.integrate import odeint
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import HTML, display
import matplotlib as mpl

# Enable inline plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

# Fix animation embed limit (increase to 50MB)
mpl.rcParams['animation.embed_limit'] = 50

# Output directory
OUTPUT_DIR = r'C:\Users\ducmi\OneDrive\Desktop\(29-12--04-01)TinhToanKhoaHocThanKinhVaUngDung\Quiz\HH-LIF-Reservoir-Simulation\Output'

print("Libraries loaded successfully")
print(f"Output directory: {OUTPUT_DIR}")

Libraries loaded successfully
Output directory: C:\Users\ducmi\OneDrive\Desktop\(29-12--04-01)TinhToanKhoaHocThanKinhVaUngDung\Quiz\HH-LIF-Reservoir-Simulation\Output


## Step 1: Define parameters & rate functions

In [9]:
# ============ PARAMETERS ============
C_m = 1.0        # Membrane capacitance (μF/cm²)
g_Na = 120.0     # Max Na conductance (mS/cm²)
g_K = 36.0       # Max K conductance (mS/cm²)
g_L = 0.3        # Leak conductance (mS/cm²)
E_Na = 50.0      # Na reversal potential (mV)
E_K = -77.0      # K reversal potential (mV)
E_L = -54.387    # Leak reversal potential (mV)
V_0 = -65.0      # Initial membrane potential (mV)
I_ext = 20.0     # External current (μA/cm²)

# ============ RATE FUNCTIONS (6 functions) ============
def alpha_m(V): return 0.1 * (V + 40.0) / (1.0 - np.exp(-(V + 40.0) / 10.0))
def beta_m(V):  return 4.0 * np.exp(-(V + 65.0) / 18.0)
def alpha_h(V): return 0.07 * np.exp(-(V + 65.0) / 20.0)
def beta_h(V):  return 1.0 / (1.0 + np.exp(-(V + 35.0) / 10.0))
def alpha_n(V): return 0.01 * (V + 55.0) / (1.0 - np.exp(-(V + 55.0) / 10.0))
def beta_n(V):  return 0.125 * np.exp(-(V + 65.0) / 80.0)

# ============ ION CURRENTS ============
def I_Na(V, m, h): return g_Na * (m**3) * h * (V - E_Na)
def I_K(V, n):     return g_K * (n**4) * (V - E_K)
def I_L(V):        return g_L * (V - E_L)

print(f"Parameters: V_0={V_0}mV, I_ext={I_ext}μA/cm²")
print(f"Defined 6 rate functions: alpha_m, beta_m, alpha_h, beta_h, alpha_n, beta_n")

Parameters: V_0=-65.0mV, I_ext=20.0μA/cm²
Defined 6 rate functions: alpha_m, beta_m, alpha_h, beta_h, alpha_n, beta_n


## Step 2: ODE system & simulation

In [10]:
def hodgkin_huxley(y, t):
    """HH ODE system: [V, m, h, n]"""
    V, m, h, n = y
    I_input = I_ext if t > 5 else 0  # Step input at t=5ms
    
    dVdt = (I_input - I_Na(V, m, h) - I_K(V, n) - I_L(V)) / C_m
    dmdt = alpha_m(V) * (1 - m) - beta_m(V) * m
    dhdt = alpha_h(V) * (1 - h) - beta_h(V) * h
    dndt = alpha_n(V) * (1 - n) - beta_n(V) * n
    return [dVdt, dmdt, dhdt, dndt]

# Initial conditions
m_0 = alpha_m(V_0) / (alpha_m(V_0) + beta_m(V_0))
h_0 = alpha_h(V_0) / (alpha_h(V_0) + beta_h(V_0))
n_0 = alpha_n(V_0) / (alpha_n(V_0) + beta_n(V_0))
y0 = [V_0, m_0, h_0, n_0]

# Time array
t = np.arange(0, 50, 0.01)

# Solve ODE
print("Running simulation...")
solution = odeint(hodgkin_huxley, y0, t)
V, m, h, n = solution.T

# Calculate currents
I_Na_array = np.array([I_Na(V[i], m[i], h[i]) for i in range(len(t))])
I_K_array = np.array([I_K(V[i], n[i]) for i in range(len(t))])

print(f"Simulation complete: {len(t)} time points")
print(f"V_peak = {V.max():.2f} mV, I_Na_min = {I_Na_array.min():.2f} μA/cm²")

Running simulation...
Simulation complete: 5000 time points
V_peak = 41.29 mV, I_Na_min = -797.44 μA/cm²


## Step 3: Static plots (V, I_Na, I_K)

In [ ]:
import os

fig, axes = plt.subplots(3, 1, figsize=(12, 9))
fig.suptitle('Hodgkin-Huxley Model: Action Potential', fontsize=16, fontweight='bold')

# V(t)
axes[0].plot(t, V, 'b-', linewidth=2, label='Membrane Potential')
axes[0].axhline(V_0, color='gray', linestyle='--', alpha=0.5, label=f'V₀={V_0}mV')
axes[0].set_ylabel('V (mV)', fontweight='bold')
axes[0].set_title('(a) Membrane Potential')
axes[0].legend()
axes[0].grid(alpha=0.3)

# I_Na(t)
axes[1].plot(t, I_Na_array, 'r-', linewidth=2, label='I_Na')
axes[1].axhline(0, color='k', linestyle='-', alpha=0.3)
axes[1].set_ylabel('I_Na (μA/cm²)', fontweight='bold')
axes[1].set_title('(b) Sodium Current')
axes[1].legend()
axes[1].grid(alpha=0.3)

# I_K(t)
axes[2].plot(t, I_K_array, 'g-', linewidth=2, label='I_K')
axes[2].axhline(0, color='k', linestyle='-', alpha=0.3)
axes[2].set_xlabel('Time (ms)', fontweight='bold')
axes[2].set_ylabel('I_K (μA/cm²)', fontweight='bold')
axes[2].set_title('(c) Potassium Current')
axes[2].legend()
axes[2].grid(alpha=0.3)

plt.tight_layout()

# Save to output directory
output_path = os.path.join(OUTPUT_DIR, 'hh_static_plots.png')
plt.savefig(output_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"Static plots saved to: {output_path}")

## Step 4: Animation - Action Potential Wave (chạy trực tiếp trong notebook)

In [ ]:
# Create animation
fig_anim, ax = plt.subplots(figsize=(10, 5))
ax.set_xlim(0, 50)
ax.set_ylim(-80, 50)
ax.set_xlabel('Time (ms)', fontweight='bold')
ax.set_ylabel('Membrane Potential (mV)', fontweight='bold')
ax.set_title('Action Potential Animation', fontweight='bold')
ax.grid(alpha=0.3)
ax.axhline(0, color='k', linestyle='-', alpha=0.3)

line, = ax.plot([], [], 'b-', linewidth=2.5)
point, = ax.plot([], [], 'ro', markersize=8)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=12, 
                    verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

def init():
    line.set_data([], [])
    point.set_data([], [])
    time_text.set_text('')
    return line, point, time_text

def animate(frame):
    # Update every 20 frames (reduced from 10 to save size)
    idx = frame * 20
    if idx >= len(t):
        idx = len(t) - 1
    
    line.set_data(t[:idx], V[:idx])
    point.set_data([t[idx]], [V[idx]])
    time_text.set_text(f't = {t[idx]:.1f} ms\nV = {V[idx]:.1f} mV')
    return line, point, time_text

print("Creating animation...")
# Reduce frames from len(t)//10 to len(t)//20 to reduce file size
anim = FuncAnimation(fig_anim, animate, init_func=init, 
                    frames=len(t)//20, interval=20, blit=True, repeat=True)

# Display animation INLINE in notebook
print("Displaying animation below...")
display(HTML(anim.to_jshtml()))
plt.close()

# Also save as GIF
print("Saving GIF...")
gif_path = os.path.join(OUTPUT_DIR, 'hh_action_potential.gif')
writer = PillowWriter(fps=30)
anim.save(gif_path, writer=writer)
print(f"Animation saved to: {gif_path}")

## Step 5: Interactive plot (Plotly - chạy trực tiếp trong notebook)

In [ ]:
# Create interactive subplot
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Membrane Potential (V)', 'Sodium Current (I_Na)', 'Potassium Current (I_K)'),
    vertical_spacing=0.1
)

# Add traces
fig.add_trace(go.Scatter(x=t, y=V, mode='lines', name='V', line=dict(color='blue', width=2)), row=1, col=1)
fig.add_trace(go.Scatter(x=t, y=I_Na_array, mode='lines', name='I_Na', line=dict(color='red', width=2)), row=2, col=1)
fig.add_trace(go.Scatter(x=t, y=I_K_array, mode='lines', name='I_K', line=dict(color='green', width=2)), row=3, col=1)

# Update layout
fig.update_xaxes(title_text="Time (ms)", row=3, col=1)
fig.update_yaxes(title_text="V (mV)", row=1, col=1)
fig.update_yaxes(title_text="Current (μA/cm²)", row=2, col=1)
fig.update_yaxes(title_text="Current (μA/cm²)", row=3, col=1)

fig.update_layout(
    height=800,
    title_text="Hodgkin-Huxley Model - Interactive Visualization",
    showlegend=True,
    hovermode='x unified'
)

# Show inline in notebook
fig.show()

# Also save HTML file
html_path = os.path.join(OUTPUT_DIR, 'hh_interactive.html')
fig.write_html(html_path)
print(f"Interactive plot saved to: {html_path}")

## Step 6: Explanation - Depolarization & Repolarization

In [14]:
# Key statistics
idx_peak = np.argmax(V)
idx_I_Na_min = np.argmin(I_Na_array)
idx_I_K_max = np.argmax(I_K_array)

print("="*70)
print("VAI TRO CUA I_Na VA I_K TRONG ACTION POTENTIAL")
print("="*70)
print()
print("KEY STATISTICS:")
print(f"   - V_peak = {V[idx_peak]:.2f} mV at t = {t[idx_peak]:.2f} ms")
print(f"   - I_Na_min = {I_Na_array[idx_I_Na_min]:.2f} μA/cm² at t = {t[idx_I_Na_min]:.2f} ms")
print(f"   - I_K_max = {I_K_array[idx_I_K_max]:.2f} μA/cm² at t = {t[idx_I_K_max]:.2f} ms")
print()
print("DEPOLARIZATION (khu cuc): t ~ 5-8 ms")
print("   -> I_Na (am, lon): Na+ do vao te bao")
print("   -> V tang nhanh: -65mV -> +40mV")
print("   -> Co che: Positive feedback (V tang -> kenh Na mo -> I_Na tang -> V tang)")
print()
print("REPOLARIZATION (tai phan cuc): t ~ 10-20 ms")
print("   -> I_K (duong, lon): K+ thoat ra ngoai te bao")
print("   -> V giam ve nghi: +40mV -> -65mV")
print("   -> Co che: Kenh K mo cham, dong cham")
print()
print("KET LUAN:")
print("   - I_Na: Nguyen nhan chinh DEPOLARIZATION (nhanh)")
print("   - I_K: Nguyen nhan chinh REPOLARIZATION (cham)")
print("="*70)

VAI TRO CUA I_Na VA I_K TRONG ACTION POTENTIAL

KEY STATISTICS:
   - V_peak = 41.29 mV at t = 6.51 ms
   - I_Na_min = -797.44 μA/cm² at t = 7.40 ms
   - I_K_max = 850.28 μA/cm² at t = 7.40 ms

DEPOLARIZATION (khu cuc): t ~ 5-8 ms
   -> I_Na (am, lon): Na+ do vao te bao
   -> V tang nhanh: -65mV -> +40mV
   -> Co che: Positive feedback (V tang -> kenh Na mo -> I_Na tang -> V tang)

REPOLARIZATION (tai phan cuc): t ~ 10-20 ms
   -> I_K (duong, lon): K+ thoat ra ngoai te bao
   -> V giam ve nghi: +40mV -> -65mV
   -> Co che: Kenh K mo cham, dong cham

KET LUAN:
   - I_Na: Nguyen nhan chinh DEPOLARIZATION (nhanh)
   - I_K: Nguyen nhan chinh REPOLARIZATION (cham)


## Summary

**Completed:**
1. Defined all parameters & 6 rate functions (alpha_m, beta_m, alpha_h, beta_h, alpha_n, beta_n)
2. Simulated with V_0 = -65 mV, I_ext = 20 μA/cm²
3. Plotted V, I_Na, I_K over time
4. Created **inline animation** (chay trong notebook) + saved GIF
5. Created **inline interactive plot** (Plotly) + saved HTML
6. Explained roles: **I_Na -> depolarization**, **I_K -> repolarization**

**Output files saved to:**  
`C:\Users\ducmi\OneDrive\Desktop\(29-12--04-01)TinhToanKhoaHocThanKinhVaUngDung\Quiz\HH-LIF-Reservoir-Simulation\Output`